# 모델 구조

In [51]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Global State

In [52]:
from langgraph.graph import MessagesState
import pandas as pd

class State(MessagesState):
    
    # 입력된 데이터의 ID
    input_id: str = ''  
    
    # 메타데이터 검색을 통해 확인한 제목과 설명
    subject: str = ''   
    description: str = ''
    
    # LLM이 생성한 검색어 리스트
    query : list[str] = []
    
    # 검색어를 통해 검색한 전체 데이터
    data: pd.DataFrame = pd.DataFrame()
    
    # 연관성 검색을 통해 확인한 topK 데이터의 ID
    relevant_id: list[str] = []


## 주제, 설명 가져오기

In [53]:
import os, requests, json

def data_input(state: State):

    API_KEY = os.getenv("DATAON_META_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_META_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/" + state["input_id"]
    params = {"key": API_KEY}

    res = requests.get(url, params=params, timeout=20)
    data = res.json()

    # json 저장
    with open("../data/input_data.json", "w", encoding="utf-8") as f:
        json.dump(data['records'], f, ensure_ascii=False, indent=4)

    subject, description = data['records']['dataset_title_etc_main'], data['records']['dataset_expl_etc_main']
    print('\n[subject]\n', subject)
    print('\n[description]\n', description)

    return {'subject':subject, 'description' : description} 

## 검색어 생성

In [54]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from typing_extensions import Annotated
from pydantic import Field, BaseModel

# Schema
class QueryResult(BaseModel):
    query: Annotated[
        list[str],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="가장 적절한 검색어들의 리스트, 길이 최소 3개/최대 5개", 
        )
    ]
    
# Prompt
query_template = '''
주어진 제목과 설명을 바탕으로, 가장 연관성이 높은 논문과 데이터셋을 검색하려 합니다.

가장 의미적으로 관련성이 높은 논문과 데이터셋을 검색할 수 있을 쿼리를 만들어 주세요.

[조건]
1. 쿼리는 2~3단어로 구성되어야 합니다.
2. 검색 방식은 쿼리와 정확히 일치하는 내용이 있는 논문이나 데이터를 반환하는 형식입니다.

[Input]
연구 주제: {subject}
연구 설명: {description}

[Output]
가장 적절한 3~5개의 쿼리를 JSON으로 출력해주세요.
'''

query_prompt = PromptTemplate.from_template(query_template)

# Node
def generate_query(state: State):

    prompt = query_prompt.invoke(
        {
            'subject': state['subject'], 
            'description': state['description'],
        }
    )

    # sLLM
    sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    structured_sllm = sllm.with_structured_output(QueryResult)
    res = structured_sllm.invoke(prompt)
    query = res.query
    print('\n[query]\n', query)

    return {'query': query}


## 쿼리에 따른 데이터 검색

In [55]:
import os, requests
import pandas as pd

def query_search(state: State):
    
    API_KEY = os.getenv("DATAON_SEARCH_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"
    print('\n[response]')
    df = pd.DataFrame()
    for query in state['query']:
        params = {"key": API_KEY, "query": query, "from": 0, "size": 10}
        # key / CHAR / 필수 / API_KEY
        # query / CHAR / 필수 / 검색키워드
        # from / CHAR / 옵션 / 페이지시작위치
        # size / CHAR / 옵션 / 페이지사이즈

        res = requests.get(url, params=params, timeout=20)
        data = res.json()
        print(data['response'])
        
        if "records" in data:
            tmp = pd.DataFrame(data["records"])
            tmp["query"] = query
            df = pd.concat([df, tmp], ignore_index=True)

    df = df.drop_duplicates(subset='svc_id')
    df.to_csv('../data/search_results.csv', index=False, encoding='utf-8')
    print('\n[total data length]\n', len(df))
    
    return {'data': df}

## 연관된 데이터 선정

In [56]:
from typing_extensions import Annotated
from pydantic import Field, BaseModel
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Schema
class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[str],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="가장 관련성이 높은 데이터 ID 리스트, 길이 최소 3개/최대 5개", 
        )
    ]

# Prompt
relevance_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 제목
- 설명

[목표]
주어진 연구 주제와 가장 관련성 높은 3~5개의 데이터 항목을 선택하세요. 

[Input]
연구 주제: {subject}
연구 설명: {description}

[Data]
데이터 목록:
{data}

[Output]
가장 관련성 높은 3~5개 항목의 svc_id를 JSON으로 출력해주세요.
'''

relevance_prompt = PromptTemplate.from_template(relevance_template)

# Node
def select_relevance(state: State):

    df, subject, description = state['data'], state['subject'], state['description']

    prompt = relevance_prompt.invoke(
        {
            'subject': subject, 
            'description': description,
            'data': df[['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main']].to_dict(orient="records"),
        }
    )

    sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    structured_sllm = sllm.with_structured_output(IDRelevance)
    res = structured_sllm.invoke(prompt)

    id_list = res.relevant_id
    print('\n[id_list]\n', id_list)
    
    return {'relevant_id': id_list}

# 그래프 구조

In [57]:
from langgraph.graph import START, END, StateGraph

def build_graph():
    builder = StateGraph(State)
    
    builder.add_sequence([data_input, generate_query, query_search, select_relevance]) 
    
    builder.add_edge(START, 'data_input')
    builder.add_edge('select_relevance', END)
    
    return builder.compile()

graph = build_graph()

In [58]:
svc_id = 'b37f0c9413eeb7c45f6fe31cbe3a41ef'
print('[input_id]\n', svc_id)

res = graph.invoke({
    'input_id': svc_id
})

df = res['data']
    
selected_data = df[df['svc_id'].isin(res['relevant_id'])]

print('\n[return]')
display(selected_data)

[input_id]
 b37f0c9413eeb7c45f6fe31cbe3a41ef

[subject]
 Architectural Urbanism: Melbourne/Seoul - KTA projects

[description]
 BACKGROUND: 'Architectural Urbanism: Melbourne/Seoul' was a two-city exhibition funded by RMIT University and the Korean National University, supported by the Australian Government through the Australian International Cultural Council. Kerstin Thompson Architecture (KTA) showed five works - Carrum Downs Police Station, MUMA Gallery, Lake Conneware House, Napier Street Housing and Royal Botanic Gardens Visitor Centre - as one of ten architectural firms (5 from Melbourne, 5 from Seoul) selected to exhibit. The exhibited work included large scale photographs, working drawings and exegetical text. CONTRIBUTION: The exhibition explored architectural approaches that worked 'within the city rather than upon it', architecture that 'intervenes and inserts, rather than overlays or eradicates'. Within this context, the public projects exhibited by KTA explore relationshi

,svc_id,ctlg_type,dataset_type,ctlg_type_pc,dataset_type_pc,dataset_pub_dt_pc,dataset_access_type_pc,file_yn_pc,dataset_cc_license_pc,dataset_main_lang_pc,...,ministry_etc,pjt_nm_kor,pjt_nm_etc,pjt_mngr_kor,pjt_mngr_etc,dataset_cntrbtr_kor,dataset_cntrbtr_etc,dataset_pblshr,dataset_pric,query
1,b37f0c9413eeb7c45f6fe31cbe3a41ef,02,02,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],Architectural Urbanism
5,90ca58cfe0896194588fe73299e585f1,02,02,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],Architectural Urbanism
6,a83eb22a4a62949d83c70662acdf439a,02,02,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],Architectural Urbanism
9,4f8e04de8044dd201965353514748c13,02,02,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],Architectural Urbanism
29,5a27d0958baf27c7372453fe940b4db0,02,02,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],public architecture
